# fz-CASMO Plugin Example Usage

This notebook demonstrates how to use the fz-CASMO plugin for CASMO5 lattice physics calculations.

The examples cover:
1. Installing dependencies (fz framework and fz-CASMO plugin)
2. Downloading example input files
3. Parsing input files to identify variables
4. Compiling input files with specific parameter values
5. Running parametric studies

**Note:** Running actual CASMO5 calculations requires a licensed copy of CASMO5 from Studsvik and the `CASMO_PATH` environment variable set.

## Setup: Install Dependencies

First, install the Funz framework and the fz-CASMO plugin.

In [ ]:
# Install fz framework from GitHub
%pip install -q git+https://github.com/Funz/fz.git

# Install fz-CASMO plugin
import fz
fz.install("CASMO")

print("✓ Dependencies installed successfully!")

## Setup: Download Example Files

Download the example input files from the GitHub repository.

In [ ]:
import os
import urllib.request

# Create directory for examples
os.makedirs("CASMO", exist_ok=True)

# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/Funz/fz-casmo/main/examples"

# Download example files
files_to_download = [
    ("CASMO/input.inp", f"{base_url}/CASMO/input.inp"),
]

print("Downloading example files...")
for local_path, url in files_to_download:
    try:
        urllib.request.urlretrieve(url, local_path)
        print(f"  ✓ Downloaded: {local_path}")
    except Exception as e:
        print(f"  ✗ Failed to download {local_path}: {e}")

print("\n✓ Example files ready!")

## Import Required Libraries

In [ ]:
import fz
import os
import tempfile

print(f"fz version: {fz.__version__ if hasattr(fz, '__version__') else 'unknown'}")

## Example 1: Parse Input File for Variables

Use `fz.fzi()` to identify variables in an input file.

In [ ]:
# Parse the input file to find variables
variables = fz.fzi(
    "CASMO/input.inp",
    "CASMO"
)

print(f"Variables found in input.inp: {list(variables.keys())}")

## Example 2: Compile Input Files

Use `fz.fzc()` to compile input files with specific parameter values. This substitutes variables with actual values.

In [ ]:
# Create temporary directory for compiled files
with tempfile.TemporaryDirectory() as tmpdir:
    # Compile input with specific values
    fz.fzc(
        "CASMO/input.inp",
        {"enrichment": 4.0, "fuel_temp": 900, "moderator_temp": 580, "burnup_steps": 50},
        "CASMO",
        output_dir=tmpdir
    )
    
    # Show compiled file
    # Find the compiled file (it may be in a subdirectory)
    for root, dirs, files in os.walk(tmpdir):
        for f in files:
            if f == "input.inp":
                compiled_file = os.path.join(root, f)
                print(f"Compiled file content (enrichment=4.0, fuel_temp=900):")
                print("=" * 70)
                with open(compiled_file, 'r') as file:
                    print(file.read())

## Example 3: Run a Parametric Study

Run parametric CASMO5 calculations with varying enrichment levels.

**Note:** This requires CASMO5 installed and `CASMO_PATH` environment variable set.

In [ ]:
# Define parameter values for parametric study
input_variables = {
    "enrichment": [3.0, 3.5, 4.0, 4.5],  # U-235 enrichment (%)
    "fuel_temp": 900,                     # Fuel temperature (K)
    "moderator_temp": 580,                # Moderator temperature (K)
    "burnup_steps": 50                    # Maximum burnup (MWd/kgU)
}

try:
    # Run parametric calculation
    results = fz.fzr(
        input_path="CASMO/input.inp",
        input_variables=input_variables,
        model="CASMO",
        calculators="localhost_CASMO",
        results_dir="results/enrichment_study"
    )
    
    # Display results
    print("\nResults:")
    print("=" * 70)
    print(results[['enrichment', 'burnup', 'k_inf', 'm2', 'status']])
    
except Exception as e:
    print(f"\nError running calculation: {e}")
    print("Note: CASMO5 must be installed with CASMO_PATH set.")

## Example 4: Visualization

If you have results, you can visualize them using matplotlib.

In [ ]:
import matplotlib.pyplot as plt

# Example visualization (using mock data if real results aren't available)
try:
    # Use actual results if available
    if 'results' in dir() and results is not None:
        enrichments = results['enrichment'].values
        k_inf_values = results['k_inf'].values
    else:
        # Use mock data for demonstration
        enrichments = [3.0, 3.5, 4.0, 4.5]
        k_inf_values = [1.22, 1.27, 1.31, 1.35]  # Example k-inf values
    
    plt.figure(figsize=(10, 6))
    plt.plot(enrichments, k_inf_values, 'o-', linewidth=2, markersize=8)
    plt.xlabel('Enrichment (%)', fontsize=12)
    plt.ylabel('k-infinity', fontsize=12)
    plt.title('CASMO5 Parametric Study: k-inf vs Enrichment', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Visualization error: {e}")